
data link:https://drive.google.com/file/d/1SBRe-fKSLxlyLVYrJEqO8V4nq2loRb3M/view?usp=sharing



import libs & data



In [1]:
from google.colab import drive
import os
from PIL import Image, UnidentifiedImageError
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from tensorflow.keras.applications import ConvNeXtBase
from sklearn.model_selection import train_test_split
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import zipfile
import gdown
# مسیر فایل ZIP در Google Drive
zip_path = '/content/drive/MyDrive/food_Images.zip'

# مسیر مقصد برای استخراج فایل‌ها
extract_dir = '/content/my_data'

# استخراج فایل‌ها
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("✅ فایل‌ها استخراج شدند در:", extract_dir)


✅ فایل‌ها استخراج شدند در: /content/my_data


# **تقسیم بندی داده**

In [3]:
def clean_directory(directory):
    for root, _, files in os.walk(directory):
        for fname in files:
            file_ = os.path.join(root, fname)
            try:
                with Image.open(file_) as img:
                    img.verify()  # Just check if it's an image
            except (UnidentifiedImageError, OSError):
                print(f"Deleting corrupted or invalid image: {file_}")
                os.remove(file_)

clean_directory("/content/my_data/train")

Deleting corrupted or invalid image: /content/my_data/train/chicken/052f1133-141.jpg
Deleting corrupted or invalid image: /content/my_data/train/chicken/6fcf7983-225.jpg
Deleting corrupted or invalid image: /content/my_data/train/chicken/2a2a3166-2d2.jpg
Deleting corrupted or invalid image: /content/my_data/train/chicken/e5574159-a60.jpg
Deleting corrupted or invalid image: /content/my_data/train/chicken/1b4ed8be-285.jpg


In [4]:
data_dir = '/content/my_data/train'
img_height, img_width = 224, 224  # ورودی استاندارد ConvNeXt
batch_size = 32
# لیبل‌ها
labels = [name for name in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, name))]
label_to_index = {label: idx for idx, label in enumerate(labels)}
num_classes = len(labels)

# خواندن تصاویر و لیبل‌ها
X, y = [], []
for label in labels:
    folder_path = os.path.join(data_dir, label)
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img = tf.keras.utils.load_img(file_path, target_size=(img_height, img_width))
            img_array = tf.keras.utils.img_to_array(img) / 255.0
            X.append(img_array)
            y.append(label_to_index[label])

X = np.array(X, dtype=np.float32)
y = to_categorical(np.array(y), num_classes=num_classes)

In [5]:
# --- تقسیم داده‌ها به train و test ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ConvNeXtTiny
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import mixed_precision

# ✅ فعال‌سازی Mixed Precision
mixed_precision.set_global_policy('mixed_float16')

# --------------------
# تنظیمات
# --------------------
data_dir = "/content/my_data/train"
img_height, img_width = 128, 128
batch_size = 128
num_classes = 22

# --------------------
# Data Augmentation + Generators
# --------------------
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',  # ✅ خروجی را one-hot می‌کند
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',  # ✅ خروجی را one-hot می‌کند
    subset='validation',
    shuffle=False
)

# --------------------
# ساخت مدل ConvNeXtTiny
# --------------------
base_model = ConvNeXtTiny(weights="imagenet", include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False  # فریز کردن مدل پایه

# Head سفارشی
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax', dtype='float32')(x)  # خروجی float32 برای جلوگیری از ناپایداری

model = Model(inputs=base_model.input, outputs=output)

# --------------------
# مرحله 1: آموزش فقط Head
# --------------------
model.compile(optimizer=Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("🔒 مرحله ۱: فقط Head آموزش داده می‌شود")
model.summary()

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, min_lr=1e-6)

history1 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=3,
    callbacks=[early_stop, reduce_lr]
)

# --------------------
# مرحله 2: Fine-tune کل مدل
# --------------------
base_model.trainable = True  # باز کردن مدل پایه

model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("🔓 مرحله ۲: کل مدل (base + head) آموزش داده می‌شود")

history2 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[early_stop, reduce_lr]
)

# --------------------
# ارزیابی نهایی
# --------------------
val_loss, val_acc = model.evaluate(val_generator)
print(f"🎯 دقت نهایی روی Validation: {val_acc:.2%}")

Found 19276 images belonging to 22 classes.
Found 4808 images belonging to 22 classes.
🔒 مرحله ۱: فقط Head آموزش داده می‌شود


Model: "functional_24"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_20      │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cast_6 (Cast)       │ (None, 128, 128,  │          0 │ input_layer_20[0… │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_pres… │ (None, 128, 128,  │          0 │ cast_6[0][0]      │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stem  │ (None, 32, 32,    │      4,896 │ convnext_tiny_pr… │
│ (Sequential)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 32, 32,    │      4,800 │ convnext_tiny_st… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 32, 32,    │        192 │ convnext_tiny_st… │
│ (LayerNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 32, 32,    │     37,248 │ convnext_tiny_st… │
│ (Dense)             │ 384)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 32, 32,    │          0 │ convnext_tiny_st… │
│ (Activation)        │ 384)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 32, 32,    │     36,960 │ convnext_tiny_st… │
│ (Dense)             │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 32, 32,    │         96 │ convnext_tiny_st… │
│ (LayerScale)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 32, 32,    │          0 │ convnext_tiny_st… │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_72 (Add)        │ (None, 32, 32,    │          0 │ convnext_tiny_st… │
│                     │ 96)               │            │ convnext_tiny_st… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 32, 32,    │      4,800 │ add_72[0][0]      │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 32, 32,    │        192 │ convnext_tiny_st… │
│ (LayerNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 32, 32,    │     37,248 │ convnext_tiny_st… │
│ (Dense)             │ 384)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 32, 32,    │          0 │ convnext_tiny_st… │
│ (Activation)        │ 384)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ convnext_tiny_stag… │ (None, 32, 32,    │     36,960 │ convnext_tiny_st

 Total params: 28,022,646 (106.90 MB)

 Trainable params: 202,518 (791.09 KB)

 Non-trainable params: 27,820,128 (106.13 MB)

Epoch 1/3
151/151 ━━━━━━━━━━━━━━━━━━━━ 7625s 50s/step - accuracy: 0.1537 - loss: 2.9836 - val_accuracy: 0.2710 - val_loss: 2.5539 - learning_rate: 0.0010
Epoch 2/3
 22/151 ━━━━━━━━━━━━━━━━━━━━ 1:26:49 40s/step - accuracy: 0.2537 - loss: 2.5741